<a href="https://colab.research.google.com/github/woo1027/chinese_bert/blob/main/chinese_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button");
     btn.click()
     }

   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

print("Done.")

from google.colab import drive
drive.mount('/content/drive')

<IPython.core.display.Javascript object>

Done.
Mounted at /content/drive


In [ ]:
import pandas as pd

import matplotlib.pyplot as plt


In [ ]:
toxic = pd.read_csv("/content/drive/My Drive//toxic.csv")
toxic.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean,clean_com_chinese
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation Why the edits made under my userna...,解釋 為什麼在我的用戶名 Hardcore Metallica Fan 下所做的編輯會被恢復...
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,D'aww! He matches this background colour I am ...,"哇哦！他與我似乎堅持的背景顏色相匹配。謝謝。談話 21 51, 2016 年 1 月 11 ..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"Hey man, I am really not trying to edit war. I...",嘿夥計，我真的不是想編輯戰爭。只是這個傢伙不斷刪除相關資訊並透過編輯而不是我的討論頁面與我交...
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,More I cannot make any real suggestions on im...,更多我無法對改進提出任何真正的建議我想知道部分統計數據是否應該稍後發布，或者“事故類型”的一...
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"You, sir, are my hero. Any chance you remember...",先生，您是我的英雄。您是否記得該頁面位於哪個頁面？


In [ ]:
print("訓練集遺失值：\n{}".format(toxic.isnull().sum()))

訓練集遺失值：
Unnamed: 0              0
id                      0
comment_text            0
toxic                   0
severe_toxic            0
obscene                 0
threat                  0
insult                  0
identity_hate           0
clean                   8
clean_com_chinese    9579
dtype: int64


In [ ]:
toxic = toxic.dropna(axis=0)


In [ ]:
newdata = toxic.drop(columns=['Unnamed: 0'])
newdata

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean,clean_com_chinese
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation Why the edits made under my userna...,解釋 為什麼在我的用戶名 Hardcore Metallica Fan 下所做的編輯會被恢復...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,D'aww! He matches this background colour I am ...,"哇哦！他與我似乎堅持的背景顏色相匹配。謝謝。談話 21 51, 2016 年 1 月 11 ..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"Hey man, I am really not trying to edit war. I...",嘿夥計，我真的不是想編輯戰爭。只是這個傢伙不斷刪除相關資訊並透過編輯而不是我的討論頁面與我交...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,More I cannot make any real suggestions on im...,更多我無法對改進提出任何真正的建議我想知道部分統計數據是否應該稍後發布，或者“事故類型”的一...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"You, sir, are my hero. Any chance you remember...",先生，您是我的英雄。您是否記得該頁面位於哪個頁面？
...,...,...,...,...,...,...,...,...,...,...
149995,6512322135cd1079,Then you should probably go to October 2014 Je...,0,0,0,0,0,0,Then you should probably go to October 2014 Je...,那麼您可能應該轉到 2014 年 10 月耶路撒冷車輛襲擊，並要求將頁面移至 2014 年 ...
149996,651462d855af1a35,You have nothing better to do than stallk and ...,0,0,0,0,0,0,You have nothing better to do than stallk and ...,除了拖延和騷擾任何與你意見不同的人之外，你沒有什麼更好的事情要做。沒有針對您的評論。正如我警...
149997,65175ab414ec6f38,This is already in the article. - Ghost,0,0,0,0,0,0,This is already in the article. Ghost,這已經在文章中了。鬼
149998,651c3da3e5310d6b,"""\n\n Annual varieties of Foxglove \n\nAlthoug...",0,0,0,0,0,0,Annual varieties of Foxglove Although the bes...,一年生品種的毛地黃 雖然最著名的毛地黃是兩年生的，換句話說，您必須等待一整年才能開花，但現在...


clean_com_chinese欄位 為 翻譯成中文的評論

In [ ]:
label_stats = newdata[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum()
print(label_stats)

comments=newdata['clean_com_chinese'].to_list()
comments[:5]

toxic            14356
severe_toxic      1514
obscene           7960
threat             445
insult            7410
identity_hate     1318
dtype: int64


['解釋 為什麼在我的用戶名 Hardcore Metallica Fan 下所做的編輯會被恢復？它們並不是故意破壞，只是在紐約玩偶 FAC 投票後關閉了一些 GA。請不要從討論頁面刪除該模板，因為我現在已經退休了。89.205.38.27',
 '哇哦！他與我似乎堅持的背景顏色相匹配。謝謝。談話 21 51, 2016 年 1 月 11 日 (UTC)',
 '嘿夥計，我真的不是想編輯戰爭。只是這個傢伙不斷刪除相關資訊並透過編輯而不是我的討論頁面與我交談。他似乎更關心格式而不是實際資訊。',
 '更多我無法對改進提出任何真正的建議我想知道部分統計數據是否應該稍後發布，或者“事故類型”的一小部分我認為參考文獻可能需要整理，以便它們都採用完全相同的格式，即日期格式等.我可以稍後做，如果沒有其他人先做的話，如果您對參考文獻的格式樣式有任何偏好或想自己做，請告訴我。待審稿的文章似乎積壓了，所以我猜審稿人出現之前可能會有延遲。它以相關形式列出，例如維基百科好文章提名運輸',
 '先生，您是我的英雄。您是否記得該頁面位於哪個頁面？']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split data into training, testing sets & validation sets
train_data, test_data, train_labels, test_labels = train_test_split(
    comments, newdata.iloc[:,2:8].values, test_size=0.3, random_state=1)

#validation set
test_data, val_data, test_labels, val_labels = train_test_split(
    test_data, test_labels, test_size=0.67, random_state=1)

print('numbers of training Dataset ',len(train_data))
print('numbers of testing Dataset',len(test_data))
print('numbers of validation Dataset',len(val_data))

numbers of training Dataset  104994
numbers of testing Dataset 14849
numbers of validation Dataset 30149


In [ ]:
def tokenize_and_encode(tokenizer, comments, labels, max_length=128):
    # Initialize empty lists to store tokenized inputs and attention masks
    input_ids = []
    attention_masks = []

    # Iterate through each comment in the 'comments' list
    for comment in comments:
        # Tokenize and encode the comment using the BERT tokenizer
        encoded_dict = tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        # Append the tokenized input and attention mask to their respective lists
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists of tokenized inputs and attention masks to PyTorch tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    # Convert the labels to a PyTorch tensor with the data type float32
    labels = torch.tensor(labels, dtype=torch.float32)

    # Return the tokenized inputs, attention masks, and labels as PyTorch tensors
    return input_ids, attention_masks, labels

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

import torch
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

model =model.to(device)

cuda


In [ ]:
# Token Initialization
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

# Model Initialization
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=6)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize and Encode the comments and labels for the training set
input_ids, attention_masks, labels = tokenize_and_encode(
    tokenizer,
    train_data,
    train_labels
)

# Step 4: Tokenize and Encode the comments and labels for the test set
test_input_ids, test_attention_masks, test_labels = tokenize_and_encode(
    tokenizer,
    test_data,
    test_labels
)

# Tokenize and Encode the comments and labels for the validation set
val_input_ids, val_attention_masks, val_labels = tokenize_and_encode(
    tokenizer,
    val_data,
    val_labels
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2700: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [18]:
# Creating DataLoader for the balanced dataset
batch_size = 32
train_dataset = TensorDataset(input_ids, attention_masks, labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#test
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#val
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [31]:
# Load the tokenizer and model from the saved directory
model_name ="/content/drive/My Drive/Saved_model"
Bert_Tokenizer = BertTokenizer.from_pretrained(model_name)
Bert_Model = BertForSequenceClassification.from_pretrained(model_name).to(device)

In [32]:
def predict_user_input(input_text, model=Bert_Model, tokenizer=Bert_Tokenizer,device=device):
    user_input = [input_text]

    user_encodings = tokenizer(user_input, truncation=True, padding=True, return_tensors="pt")

    user_dataset = TensorDataset(user_encodings['input_ids'], user_encodings['attention_mask'])

    user_loader = DataLoader(user_dataset, batch_size=1, shuffle=False)

    model.eval()
    with torch.no_grad():
        for batch in user_loader:
            input_ids, attention_mask = [t.to(device) for t in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.sigmoid(logits)
    predicted_labels = (predictions.cpu().numpy() > 0.5).astype(int)
    return predicted_labels[0].tolist()

In [33]:
text = '你是智障嗎?'
predict_user_input(input_text=text)

[1, 0, 0, 0, 0, 0]

有毒的、嚴重有毒、猥褻、威脅、侮辱、身分仇恨

In [34]:
text = "你真的很討厭，但我愛你"
predict_user_input(model=Bert_Model,
                   tokenizer=Bert_Tokenizer,
                   input_text=text,
                   device=device)

[0, 0, 0, 0, 0, 0]

有毒的、嚴重有毒、猥褻、威脅、侮辱、身分仇恨

In [35]:
input_text = '你這個婊子'
predict_user_input(input_text)

[1, 1, 1, 0, 1, 0]

有毒的、嚴重有毒、猥褻、威脅、侮辱、身分仇恨

In [36]:
text = "閉嘴，笨蛋"
predict_user_input(model=Bert_Model,
                   tokenizer=Bert_Tokenizer,
                   input_text=text,
                   device=device)

[1, 0, 1, 0, 1, 0]

有毒的、嚴重有毒、猥褻、威脅、侮辱、身分仇恨

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.7 MB/s eta 0:00:00


2025-05-09 04:36:56.134 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:36:56.322 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-09 04:36:56.323 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:36:56.326 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:36:56.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:36:56.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:36:56.331 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:36:56.332 Session state does not 

2025-05-09 04:37:09.666 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:37:09.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:37:09.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:37:09.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 04:37:09.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
